In [1]:
import pandas as pd
import numpy as np

# cross-validation 
from sklearn.model_selection import train_test_split, StratifiedKFold

# classification models 
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve 
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.svm import SVC

# cox model 
from lifelines import CoxPHFitter
from lifelines.utils import concordance_index

# mlp
from sklearn.neural_network import MLPClassifier

/Users/oliviazhang/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/oliviazhang/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [3]:
data_path = './heart_failure_dataset.csv' # This line assumes the csv file to be placed in the current working directory 
                                          # To be changed if it was placed somewhere else 
df = pd.read_csv(data_path, index_col=False)
df

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,41,0,148,0,40,0,374000.00,0.80,140,1,1,68,0
1,66,0,434,1,24,1,268112.43,1.21,135,1,1,138,1
2,70,0,93,0,35,0,185000.00,1.10,134,1,1,208,0
3,72,0,140,1,50,0,216218.24,0.98,134,1,0,32,0
4,60,0,235,1,38,0,329000.00,3.00,142,0,0,30,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,55,0,748,0,45,0,263000.00,1.30,137,1,0,88,0
896,44,0,582,1,30,1,263358.03,1.60,130,1,1,244,0
897,70,0,838,1,35,1,304117.75,0.80,133,1,0,144,1
898,77,0,107,0,50,1,405514.68,1.11,137,1,1,209,1


In [3]:
df = df.rename(columns={'DEATH_EVENT': 'event', 'time': 'time'})

# Prepare the input features and target variable
X = df.drop(columns=['time', 'event'])
y = df['event']  # Target variable (renamed DEATH_EVENT)
time = df['time']
y_with_time = pd.DataFrame({'event': y, 'time': time})

# Performing an 80/20 train-test split, stratified by 'event'
X_dev, X_test, y_dev_with_time, y_test_with_time = train_test_split(X, y_with_time, test_size=0.2, stratify=y, random_state=42)
y_dev = y_dev_with_time['event']
time_dev = y_dev_with_time['time']
y_test = y_test_with_time['event']
time_test = y_test_with_time['time']

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


# Logistic regression with l2 (ridge) regularization

In [28]:
# Track the best results
best_auroc_mean = 0
best_auroc_std = 0
best_metrics = None
best_hyperparams = None

for C in [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]:
    print(f"Regularization strength: {C}")
    # Initialize lists to store the metrics for each fold
    accuracy_scores = []
    precision_scores = []
    recall_scores = []
    f1_scores = []
    auroc_scores = []
    auprc_scores = []
    # Perform 5-fold cross-validation
    for train_index, val_index in kf.split(X_dev, y_dev):
        # get train and val data 
        X_train_fold, X_val_fold = X_dev.iloc[train_index], X_dev.iloc[val_index]
        y_train_fold, y_val_fold = y_dev.iloc[train_index], y_dev.iloc[val_index]

        # Normalize the training set and apply the same params to val
        scaler = StandardScaler()
        X_train_fold_scaled = scaler.fit_transform(X_train_fold)
        X_val_fold_scaled = scaler.transform(X_val_fold)

        # fit logistic regression model 
        log_reg = LogisticRegression(max_iter=1000, random_state=42, C=C, penalty='l2')
        log_reg.fit(X_train_fold_scaled, y_train_fold)

        # evaluate on val 
        y_val_pred = log_reg.predict(X_val_fold_scaled)
        y_val_prob = log_reg.predict_proba(X_val_fold_scaled)[:, 1]  # Probabilities for positive class

        # Calculate and store classification metrics
        accuracy_scores.append(accuracy_score(y_val_fold, y_val_pred))
        precision_scores.append(precision_score(y_val_fold, y_val_pred))
        recall_scores.append(recall_score(y_val_fold, y_val_pred))
        f1_scores.append(f1_score(y_val_fold, y_val_pred))
        auroc_scores.append(roc_auc_score(y_val_fold, y_val_prob))
        auprc_scores.append(average_precision_score(y_val_fold, y_val_prob))

    # Calculate mean and standard deviation of AUROC
    auroc_mean = np.mean(auroc_scores)
    auroc_std = np.std(auroc_scores)

    # Check if this is the best AUROC so far
    if auroc_mean > best_auroc_mean:
        best_auroc_mean = auroc_mean
        best_auroc_std = auroc_std
        best_metrics = {
            'accuracy': (np.mean(accuracy_scores), np.std(accuracy_scores)),
            'precision': (np.mean(precision_scores), np.std(precision_scores)),
            'recall': (np.mean(recall_scores), np.std(recall_scores)),
            'f1_score': (np.mean(f1_scores), np.std(f1_scores)),
            'auroc': (auroc_mean, auroc_std),
            'auprc': (np.mean(auprc_scores), np.std(auprc_scores))
        }
        best_hyperparams = C

# Print the best results
print(f"Best hyperparams: {best_hyperparams}")
print(f"Best AUROC: {best_metrics['auroc'][0]:.4f} ({best_metrics['auroc'][1]:.4f})")
print(f"Best hyperparameters: C = {best_hyperparams}")
print(f"Accuracy: Mean = {best_metrics['accuracy'][0]:.4f} ({best_metrics['accuracy'][1]:.4f})")
print(f"Precision: Mean = {best_metrics['precision'][0]:.4f} ({best_metrics['precision'][1]:.4f})")
print(f"Recall: Mean = {best_metrics['recall'][0]:.4f} ({best_metrics['recall'][1]:.4f})")
print(f"F1-Score: Mean = {best_metrics['f1_score'][0]:.4f} ({best_metrics['f1_score'][1]:.4f})")
print(f"AUROC: Mean = {best_metrics['auroc'][0]:.4f} ({best_metrics['auroc'][1]:.4f})")
print(f"AUPRC: Mean = {best_metrics['auprc'][0]:.4f} ({best_metrics['auprc'][1]:.4f})")

Regularization strength: 0.0001
Regularization strength: 0.001
Regularization strength: 0.01
Regularization strength: 0.1
Regularization strength: 1
Regularization strength: 10
Regularization strength: 100
Best hyperparams: 0.001
Best AUROC: 0.6862 (0.0249)
Best hyperparameters: C = 0.001
Accuracy: Mean = 0.6292 (0.0136)
Precision: Mean = 0.6194 (0.0120)
Recall: Mean = 0.6859 (0.0222)
F1-Score: Mean = 0.6509 (0.0147)
AUROC: Mean = 0.6862 (0.0249)
AUPRC: Mean = 0.6911 (0.0359)


In [29]:
# After determining the best hyperparameter 'C' from cross-validation:
best_C = best_hyperparams  # This is the best 'C' found during cross-validation

# Normalize the entire development set and the test set
scaler = StandardScaler()
X_dev_scaled = scaler.fit_transform(X_dev)
X_test_scaled = scaler.transform(X_test)

# Prepare development and test data
train_data = pd.DataFrame(X_dev_scaled, columns=X.columns)
train_data['time'] = time_dev.values
train_data['event'] = y_dev.values
test_data = pd.DataFrame(X_test_scaled, columns=X.columns)
test_data['time'] = time_test.values
test_data['event'] = y_test.values

# fit logistic regression model 
log_reg = LogisticRegression(max_iter=1000, random_state=42, C=C, penalty='l2')
log_reg.fit(X_dev_scaled, y_dev)

# evaluate on val 
y_test_pred = log_reg.predict(X_test_scaled)
y_test_prob = log_reg.predict_proba(X_test_scaled)[:, 1]

# Calculate ROC curve and Youden's index for the test set
fpr, tpr, thresholds = roc_curve(y_test, y_test_prob)

# Calculate classification metrics on the test set
accuracy_test = accuracy_score(y_test, y_test_pred)
precision_test = precision_score(y_test, y_test_pred)
recall_test = recall_score(y_test, y_test_pred)
f1_test = f1_score(y_test, y_test_pred)
auroc_test = roc_auc_score(y_test, y_test_prob)
auprc_test = average_precision_score(y_test, y_test_prob)

# Print the test set performance
print(f"Accuracy on test set: {accuracy_test:.4f}")
print(f"Precision on test set: {precision_test:.4f}")
print(f"Recall on test set: {recall_test:.4f}")
print(f"F1-Score on test set: {f1_test:.4f}")
print(f"AUROC on test set: {auroc_test:.4f}")
print(f"AUPRC on test set: {auprc_test:.4f}")

Accuracy on test set: 0.7111
Precision on test set: 0.7349
Recall on test set: 0.6703
F1-Score on test set: 0.7011
AUROC on test set: 0.7715
AUPRC on test set: 0.7541


# Logistic regression with l1 (LASSO) regularization

In [45]:
# Track the best results
best_auroc_mean = 0
best_auroc_std = 0
best_metrics = None
best_hyperparams = None

for C in [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]:
    print(f"Regularization strength: {C}")
    # Initialize lists to store the metrics for each fold
    accuracy_scores = []
    precision_scores = []
    recall_scores = []
    f1_scores = []
    auroc_scores = []
    auprc_scores = []
    # Perform 5-fold cross-validation
    for train_index, val_index in kf.split(X_dev, y_dev):
        # get train and val data 
        X_train_fold, X_val_fold = X_dev.iloc[train_index], X_dev.iloc[val_index]
        y_train_fold, y_val_fold = y_dev.iloc[train_index], y_dev.iloc[val_index]

        # Normalize the training set and apply the same params to val
        scaler = StandardScaler()
        X_train_fold_scaled = scaler.fit_transform(X_train_fold)
        X_val_fold_scaled = scaler.transform(X_val_fold)

        # fit logistic regression model 
        log_reg = LogisticRegression(max_iter=1000, random_state=42, C=C, penalty='l1', solver='saga')
        log_reg.fit(X_train_fold_scaled, y_train_fold)

        # evaluate on val 
        y_val_pred = log_reg.predict(X_val_fold_scaled)
        y_val_prob = log_reg.predict_proba(X_val_fold_scaled)[:, 1]  # Probabilities for positive class

        # Calculate and store classification metrics
        accuracy_scores.append(accuracy_score(y_val_fold, y_val_pred))
        precision_scores.append(precision_score(y_val_fold, y_val_pred))
        recall_scores.append(recall_score(y_val_fold, y_val_pred))
        f1_scores.append(f1_score(y_val_fold, y_val_pred))
        auroc_scores.append(roc_auc_score(y_val_fold, y_val_prob))
        auprc_scores.append(average_precision_score(y_val_fold, y_val_prob))

    # Calculate mean and standard deviation of AUROC
    auroc_mean = np.mean(auroc_scores)
    auroc_std = np.std(auroc_scores)

    # Check if this is the best AUROC so far
    if auroc_mean > best_auroc_mean:
        best_auroc_mean = auroc_mean
        best_auroc_std = auroc_std
        best_metrics = {
            'accuracy': (np.mean(accuracy_scores), np.std(accuracy_scores)),
            'precision': (np.mean(precision_scores), np.std(precision_scores)),
            'recall': (np.mean(recall_scores), np.std(recall_scores)),
            'f1_score': (np.mean(f1_scores), np.std(f1_scores)),
            'auroc': (auroc_mean, auroc_std),
            'auprc': (np.mean(auprc_scores), np.std(auprc_scores))
        }
        best_hyperparams = C

# Print the best results
print(f"Best hyperparams: {best_hyperparams}")
print(f"Best AUROC: {best_metrics['auroc'][0]:.4f} ({best_metrics['auroc'][1]:.4f})")
print(f"Best hyperparameters: C = {best_hyperparams}")
print(f"Accuracy: Mean = {best_metrics['accuracy'][0]:.4f} ({best_metrics['accuracy'][1]:.4f})")
print(f"Precision: Mean = {best_metrics['precision'][0]:.4f} ({best_metrics['precision'][1]:.4f})")
print(f"Recall: Mean = {best_metrics['recall'][0]:.4f} ({best_metrics['recall'][1]:.4f})")
print(f"F1-Score: Mean = {best_metrics['f1_score'][0]:.4f} ({best_metrics['f1_score'][1]:.4f})")
print(f"AUROC: Mean = {best_metrics['auroc'][0]:.4f} ({best_metrics['auroc'][1]:.4f})")
print(f"AUPRC: Mean = {best_metrics['auprc'][0]:.4f} ({best_metrics['auprc'][1]:.4f})")

Regularization strength: 0.0001
Regularization strength: 0.001
Regularization strength: 0.01
Regularization strength: 0.1
Regularization strength: 1
Regularization strength: 10


/Users/oliviazhang/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/oliviazhang/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/oliviazhang/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/oliviazhang/opt/anaconda3/lib/python3.9/site-packages/skl

Regularization strength: 100
Best hyperparams: 100
Best AUROC: 0.6821 (0.0220)
Best hyperparameters: C = 100
Accuracy: Mean = 0.6292 (0.0231)
Precision: Mean = 0.6324 (0.0234)
Recall: Mean = 0.6336 (0.0390)
F1-Score: Mean = 0.6324 (0.0248)
AUROC: Mean = 0.6821 (0.0220)
AUPRC: Mean = 0.6814 (0.0289)


In [47]:
# After determining the best hyperparameter 'C' from cross-validation:
best_C = best_hyperparams  # This is the best 'C' found during cross-validation

# Normalize the entire development set and the test set
scaler = StandardScaler()
X_dev_scaled = scaler.fit_transform(X_dev)
X_test_scaled = scaler.transform(X_test)

# Prepare development and test data
train_data = pd.DataFrame(X_dev_scaled, columns=X.columns)
train_data['time'] = time_dev.values
train_data['event'] = y_dev.values
test_data = pd.DataFrame(X_test_scaled, columns=X.columns)
test_data['time'] = time_test.values
test_data['event'] = y_test.values

# fit logistic regression model 
log_reg = LogisticRegression(max_iter=1000, random_state=42, C=C, penalty='l2')
log_reg.fit(X_dev_scaled, y_dev)

# evaluate on val 
y_test_pred = log_reg.predict(X_test_scaled)
y_test_prob = log_reg.predict_proba(X_test_scaled)[:, 1]

# Calculate ROC curve and Youden's index for the test set
fpr, tpr, thresholds = roc_curve(y_test, y_test_prob)

# Calculate classification metrics on the test set
accuracy_test = accuracy_score(y_test, y_test_pred)
precision_test = precision_score(y_test, y_test_pred)
recall_test = recall_score(y_test, y_test_pred)
f1_test = f1_score(y_test, y_test_pred)
auroc_test = roc_auc_score(y_test, y_test_prob)
auprc_test = average_precision_score(y_test, y_test_prob)

# Print the test set performance
print(f"Accuracy on test set: {accuracy_test:.4f}")
print(f"Precision on test set: {precision_test:.4f}")
print(f"Recall on test set: {recall_test:.4f}")
print(f"F1-Score on test set: {f1_test:.4f}")
print(f"AUROC on test set: {auroc_test:.4f}")
print(f"AUPRC on test set: {auprc_test:.4f}")

Accuracy on test set: 0.7111
Precision on test set: 0.7349
Recall on test set: 0.6703
F1-Score on test set: 0.7011
AUROC on test set: 0.7715
AUPRC on test set: 0.7541


# Cox model with ridge regression

In [40]:
C_values = [0, 0.00001, 0.0001, 0.001, 0.01, 0.1, 0.5, 1, 10, 100, 1000, 10000]

# Track the best results
best_auroc_mean = 0
best_auroc_std = 0
best_metrics = None
best_hyperparams = None

# Perform grid search over C values
for C in C_values:
    print(f"Regularization strength: {C}")
    
    # Initialize lists to store the metrics for each fold
    accuracy_scores = []
    precision_scores = []
    recall_scores = []
    f1_scores = []
    auroc_scores = []
    auprc_scores = []
    concordance_indices = []

    # Perform 5-fold cross-validation
    for train_index, val_index in kf.split(X_dev, y_dev):
        # Get train and validation data
        X_train_fold, X_val_fold = X_dev.iloc[train_index], X_dev.iloc[val_index]
        y_train_fold, y_val_fold = y_dev.iloc[train_index], y_dev.iloc[val_index]
        time_train_fold, time_val_fold = time_dev.iloc[train_index], time_dev.iloc[val_index]

        # Normalize the training set and apply the same params to val
        scaler = StandardScaler()
        X_train_fold_scaled = scaler.fit_transform(X_train_fold)
        X_val_fold_scaled = scaler.transform(X_val_fold)

        # Prepare train and validation data
        train_data = pd.DataFrame(X_train_fold_scaled, columns=X.columns)
        train_data['time'] = time_train_fold.values
        train_data['event'] = y_train_fold.values
        val_data = pd.DataFrame(X_val_fold_scaled, columns=X.columns)
        val_data['time'] = time_val_fold.values
        val_data['event'] = y_val_fold.values

        # Fit Cox Proportional Hazards model
        cox_model = CoxPHFitter(penalizer=C, l1_ratio=0)  # Regularization using penalizer
        cox_model.fit(train_data, duration_col='time', event_col='event')

        # Predict the partial hazard for the validation set
        val_hazards = cox_model.predict_partial_hazard(val_data)

        # Evaluate using concordance index
        concordance_index_val = concordance_index(val_data['time'], -val_hazards, event_observed=val_data['event'])
        concordance_indices.append(concordance_index_val)

        # Calculate ROC curve and thresholds
        fpr, tpr, thresholds = roc_curve(y_val_fold, val_hazards)

        # Calculate the Youden index (TPR - FPR)
        youden_index = tpr - fpr

        # Find the index of the maximum Youden index
        optimal_threshold_index = np.argmax(youden_index)

        # Get the optimal threshold
        optimal_threshold = thresholds[optimal_threshold_index]

        # Classify the patients based on the optimal threshold
        y_val_pred = (val_hazards >= optimal_threshold).astype(int)

        # Calculate and store classification metrics
        accuracy_scores.append(accuracy_score(y_val_fold, y_val_pred))
        precision_scores.append(precision_score(y_val_fold, y_val_pred))
        recall_scores.append(recall_score(y_val_fold, y_val_pred))
        f1_scores.append(f1_score(y_val_fold, y_val_pred))
        auroc_scores.append(roc_auc_score(y_val_fold, val_hazards))
        auprc_scores.append(average_precision_score(y_val_fold, val_hazards))

    # Calculate mean and standard deviation of AUROC
    auroc_mean = np.mean(auroc_scores)
    auroc_std = np.std(auroc_scores)

    # Check if this is the best AUROC so far
    if auroc_mean > best_auroc_mean:
        best_auroc_mean = auroc_mean
        best_auroc_std = auroc_std
        best_metrics = {
            'accuracy': (np.mean(accuracy_scores), np.std(accuracy_scores)),
            'precision': (np.mean(precision_scores), np.std(precision_scores)),
            'recall': (np.mean(recall_scores), np.std(recall_scores)),
            'f1_score': (np.mean(f1_scores), np.std(f1_scores)),
            'auroc': (auroc_mean, auroc_std),
            'auprc': (np.mean(auprc_scores), np.std(auprc_scores))
        }
        best_hyperparams = C

# Print the best results
print(f"Best AUROC: {best_metrics['auroc'][0]:.4f} ({best_metrics['auroc'][1]:.4f})")
print(f"Best hyperparameters: C = {best_hyperparams}")
print(f"Accuracy: Mean = {best_metrics['accuracy'][0]:.4f} ({best_metrics['accuracy'][1]:.4f})")
print(f"Precision: Mean = {best_metrics['precision'][0]:.4f} ({best_metrics['precision'][1]:.4f})")
print(f"Recall: Mean = {best_metrics['recall'][0]:.4f} ({best_metrics['recall'][1]:.4f})")
print(f"F1-Score: Mean = {best_metrics['f1_score'][0]:.4f} ({best_metrics['f1_score'][1]:.4f})")
print(f"AUROC: Mean = {best_metrics['auroc'][0]:.4f} ({best_metrics['auroc'][1]:.4f})")
print(f"AUPRC: Mean = {best_metrics['auprc'][0]:.4f} ({best_metrics['auprc'][1]:.4f})")


Regularization strength: 0
Regularization strength: 1e-05
Regularization strength: 0.0001
Regularization strength: 0.001
Regularization strength: 0.01
Regularization strength: 0.1
Regularization strength: 0.5
Regularization strength: 1
Regularization strength: 10
Regularization strength: 100
Regularization strength: 1000
Regularization strength: 10000
Best AUROC: 0.6864 (0.0178)
Best hyperparameters: C = 1
Accuracy: Mean = 0.6653 (0.0254)
Precision: Mean = 0.6793 (0.0556)
Recall: Mean = 0.6721 (0.1353)
F1-Score: Mean = 0.6625 (0.0594)
AUROC: Mean = 0.6864 (0.0178)
AUPRC: Mean = 0.6855 (0.0258)


In [41]:
# After determining the best hyperparameter 'C' from cross-validation:
best_C = best_hyperparams  # This is the best 'C' found during cross-validation

# Normalize the entire development set and the test set
scaler = StandardScaler()
X_dev_scaled = scaler.fit_transform(X_dev)
X_test_scaled = scaler.transform(X_test)

# Prepare development and test data
train_data = pd.DataFrame(X_dev_scaled, columns=X.columns)
train_data['time'] = time_dev.values
train_data['event'] = y_dev.values
test_data = pd.DataFrame(X_test_scaled, columns=X.columns)
test_data['time'] = time_test.values
test_data['event'] = y_test.values

# Fit the Cox model using the best regularization parameter found on the entire development set
cox_model = CoxPHFitter(penalizer=best_C, l1_ratio=0)
cox_model.fit(train_data, duration_col='time', event_col='event')

# Predict the partial hazard for the test set
test_hazards = cox_model.predict_partial_hazard(test_data)

# Evaluate using the concordance index on the test set
concordance_index_test = concordance_index(test_data['time'], -test_hazards, event_observed=test_data['event'])
print(f"Concordance Index on test set: {concordance_index_test:.4f}")

# Calculate ROC curve and Youden's index for the test set
fpr, tpr, thresholds = roc_curve(y_test, test_hazards)

# Calculate the Youden index (TPR - FPR)
youden_index = tpr - fpr

# Find the index of the maximum Youden index
optimal_threshold_index = np.argmax(youden_index)

# Get the optimal threshold for binary classification
optimal_threshold = thresholds[optimal_threshold_index]

# Classify the patients based on the optimal threshold
y_test_pred = (test_hazards >= optimal_threshold).astype(int)

# Calculate classification metrics on the test set
accuracy_test = accuracy_score(y_test, y_test_pred)
precision_test = precision_score(y_test, y_test_pred)
recall_test = recall_score(y_test, y_test_pred)
f1_test = f1_score(y_test, y_test_pred)
auroc_test = roc_auc_score(y_test, test_hazards)
auprc_test = average_precision_score(y_test, test_hazards)

# Print the test set performance
print(f"Accuracy on test set: {accuracy_test:.4f}")
print(f"Precision on test set: {precision_test:.4f}")
print(f"Recall on test set: {recall_test:.4f}")
print(f"F1-Score on test set: {f1_test:.4f}")
print(f"AUROC on test set: {auroc_test:.4f}")
print(f"AUPRC on test set: {auprc_test:.4f}")

Concordance Index on test set: 0.7244
Accuracy on test set: 0.7500
Precision on test set: 0.7674
Recall on test set: 0.7253
F1-Score on test set: 0.7458
AUROC on test set: 0.7921
AUPRC on test set: 0.7714


# Cox model with LASSO

In [50]:
C_values = [0, 0.00001, 0.0001, 0.001, 0.01, 0.1, 0.5, 1, 10, 100, 1000, 10000]

# Track the best results
best_auroc_mean = 0
best_auroc_std = 0
best_metrics = None
best_hyperparams = None

# Perform grid search over C values
for C in C_values:
    print(f"Regularization strength: {C}")
    
    # Initialize lists to store the metrics for each fold
    accuracy_scores = []
    precision_scores = []
    recall_scores = []
    f1_scores = []
    auroc_scores = []
    auprc_scores = []
    concordance_indices = []

    # Perform 5-fold cross-validation
    for train_index, val_index in kf.split(X_dev, y_dev):
        # Get train and validation data
        X_train_fold, X_val_fold = X_dev.iloc[train_index], X_dev.iloc[val_index]
        y_train_fold, y_val_fold = y_dev.iloc[train_index], y_dev.iloc[val_index]
        time_train_fold, time_val_fold = time_dev.iloc[train_index], time_dev.iloc[val_index]

        # Normalize the training set and apply the same params to val
        scaler = StandardScaler()
        X_train_fold_scaled = scaler.fit_transform(X_train_fold)
        X_val_fold_scaled = scaler.transform(X_val_fold)

        # Prepare train and validation data
        train_data = pd.DataFrame(X_train_fold_scaled, columns=X.columns)
        train_data['time'] = time_train_fold.values
        train_data['event'] = y_train_fold.values
        val_data = pd.DataFrame(X_val_fold_scaled, columns=X.columns)
        val_data['time'] = time_val_fold.values
        val_data['event'] = y_val_fold.values

        # Fit Cox Proportional Hazards model
        cox_model = CoxPHFitter(penalizer=C, l1_ratio=1)  # Regularization using penalizer
        cox_model.fit(train_data, duration_col='time', event_col='event')

        # Predict the partial hazard for the validation set
        val_hazards = cox_model.predict_partial_hazard(val_data)

        # Evaluate using concordance index
        concordance_index_val = concordance_index(val_data['time'], -val_hazards, event_observed=val_data['event'])
        concordance_indices.append(concordance_index_val)

        # Calculate ROC curve and thresholds
        fpr, tpr, thresholds = roc_curve(y_val_fold, val_hazards)

        # Calculate the Youden index (TPR - FPR)
        youden_index = tpr - fpr

        # Find the index of the maximum Youden index
        optimal_threshold_index = np.argmax(youden_index)

        # Get the optimal threshold
        optimal_threshold = thresholds[optimal_threshold_index]

        # Classify the patients based on the optimal threshold
        y_val_pred = (val_hazards >= optimal_threshold).astype(int)

        # Calculate and store classification metrics
        accuracy_scores.append(accuracy_score(y_val_fold, y_val_pred))
        precision_scores.append(precision_score(y_val_fold, y_val_pred))
        recall_scores.append(recall_score(y_val_fold, y_val_pred))
        f1_scores.append(f1_score(y_val_fold, y_val_pred))
        auroc_scores.append(roc_auc_score(y_val_fold, val_hazards))
        auprc_scores.append(average_precision_score(y_val_fold, val_hazards))

    # Calculate mean and standard deviation of AUROC
    auroc_mean = np.mean(auroc_scores)
    auroc_std = np.std(auroc_scores)

    # Check if this is the best AUROC so far
    if auroc_mean > best_auroc_mean:
        best_auroc_mean = auroc_mean
        best_auroc_std = auroc_std
        best_metrics = {
            'accuracy': (np.mean(accuracy_scores), np.std(accuracy_scores)),
            'precision': (np.mean(precision_scores), np.std(precision_scores)),
            'recall': (np.mean(recall_scores), np.std(recall_scores)),
            'f1_score': (np.mean(f1_scores), np.std(f1_scores)),
            'auroc': (auroc_mean, auroc_std),
            'auprc': (np.mean(auprc_scores), np.std(auprc_scores))
        }
        best_hyperparams = C

# Print the best results
print(f"Best AUROC: {best_metrics['auroc'][0]:.4f} ({best_metrics['auroc'][1]:.4f})")
print(f"Best hyperparameters: C = {best_hyperparams}")
print(f"Accuracy: Mean = {best_metrics['accuracy'][0]:.4f} ({best_metrics['accuracy'][1]:.4f})")
print(f"Precision: Mean = {best_metrics['precision'][0]:.4f} ({best_metrics['precision'][1]:.4f})")
print(f"Recall: Mean = {best_metrics['recall'][0]:.4f} ({best_metrics['recall'][1]:.4f})")
print(f"F1-Score: Mean = {best_metrics['f1_score'][0]:.4f} ({best_metrics['f1_score'][1]:.4f})")
print(f"AUROC: Mean = {best_metrics['auroc'][0]:.4f} ({best_metrics['auroc'][1]:.4f})")
print(f"AUPRC: Mean = {best_metrics['auprc'][0]:.4f} ({best_metrics['auprc'][1]:.4f})")


Regularization strength: 0
Regularization strength: 1e-05
Regularization strength: 0.0001
Regularization strength: 0.001
Regularization strength: 0.01
Regularization strength: 0.1
Regularization strength: 0.5
Regularization strength: 1
Regularization strength: 10
Regularization strength: 100
Regularization strength: 1000
Regularization strength: 10000
Best AUROC: 0.6855 (0.0205)
Best hyperparameters: C = 1
Accuracy: Mean = 0.6597 (0.0191)
Precision: Mean = 0.7237 (0.0747)
Recall: Mean = 0.5760 (0.1788)
F1-Score: Mean = 0.6151 (0.0888)
AUROC: Mean = 0.6855 (0.0205)
AUPRC: Mean = 0.6883 (0.0270)


In [49]:
# After determining the best hyperparameter 'C' from cross-validation:
best_C = best_hyperparams  # This is the best 'C' found during cross-validation

# Normalize the entire development set and the test set
scaler = StandardScaler()
X_dev_scaled = scaler.fit_transform(X_dev)
X_test_scaled = scaler.transform(X_test)

# Prepare development and test data
train_data = pd.DataFrame(X_dev_scaled, columns=X.columns)
train_data['time'] = time_dev.values
train_data['event'] = y_dev.values
test_data = pd.DataFrame(X_test_scaled, columns=X.columns)
test_data['time'] = time_test.values
test_data['event'] = y_test.values

# Fit the Cox model using the best regularization parameter found on the entire development set
cox_model = CoxPHFitter(penalizer=best_C, l1_ratio=1)
cox_model.fit(train_data, duration_col='time', event_col='event')

# Predict the partial hazard for the test set
test_hazards = cox_model.predict_partial_hazard(test_data)

# Evaluate using the concordance index on the test set
concordance_index_test = concordance_index(test_data['time'], -test_hazards, event_observed=test_data['event'])
print(f"Concordance Index on test set: {concordance_index_test:.4f}")

# Calculate ROC curve and Youden's index for the test set
fpr, tpr, thresholds = roc_curve(y_test, test_hazards)

# Calculate the Youden index (TPR - FPR)
youden_index = tpr - fpr

# Find the index of the maximum Youden index
optimal_threshold_index = np.argmax(youden_index)

# Get the optimal threshold for binary classification
optimal_threshold = thresholds[optimal_threshold_index]

# Classify the patients based on the optimal threshold
y_test_pred = (test_hazards >= optimal_threshold).astype(int)

# Calculate classification metrics on the test set
accuracy_test = accuracy_score(y_test, y_test_pred)
precision_test = precision_score(y_test, y_test_pred)
recall_test = recall_score(y_test, y_test_pred)
f1_test = f1_score(y_test, y_test_pred)
auroc_test = roc_auc_score(y_test, test_hazards)
auprc_test = average_precision_score(y_test, test_hazards)

# Print the test set performance
print(f"Accuracy on test set: {accuracy_test:.4f}")
print(f"Precision on test set: {precision_test:.4f}")
print(f"Recall on test set: {recall_test:.4f}")
print(f"F1-Score on test set: {f1_test:.4f}")
print(f"AUROC on test set: {auroc_test:.4f}")
print(f"AUPRC on test set: {auprc_test:.4f}")

Concordance Index on test set: 0.7218
Accuracy on test set: 0.7556
Precision on test set: 0.7423
Recall on test set: 0.7912
F1-Score on test set: 0.7660
AUROC on test set: 0.7906
AUPRC on test set: 0.7700


# RBF SVM

In [31]:
# Initialize StratifiedKFold for cross-validation
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Define hyperparameters to tune (C and gamma for RBF kernel)
C_values = [0.01, 0.1, 1, 10, 100]
gammas = ['scale', 'auto']  # Gamma options for the RBF kernel

# Track the best results
best_auroc_mean = 0
best_auroc_std = 0
best_metrics = None
best_hyperparams = None

# Perform grid search over C and gamma
for C in C_values:
    for gamma in gammas:
        print(f"Regularization strength (C): {C}, Gamma: {gamma}")
        
        # Initialize lists to store metrics for each fold
        accuracy_scores = []
        precision_scores = []
        recall_scores = []
        f1_scores = []
        auroc_scores = []
        auprc_scores = []

        # Perform 5-fold cross-validation
        for train_index, val_index in kf.split(X_dev, y_dev):
            # Get train and validation data
            X_train_fold, X_val_fold = X_dev.iloc[train_index], X_dev.iloc[val_index]
            y_train_fold, y_val_fold = y_dev.iloc[train_index], y_dev.iloc[val_index]

            # Normalize the training set and apply the same params to validation set
            scaler = StandardScaler()
            X_train_fold_scaled = scaler.fit_transform(X_train_fold)
            X_val_fold_scaled = scaler.transform(X_val_fold)

            # Fit SVM model with RBF kernel
            svm_model = SVC(C=C, kernel='rbf', gamma=gamma, probability=True, random_state=42)
            svm_model.fit(X_train_fold_scaled, y_train_fold)

            # Predict on validation set
            y_val_pred = svm_model.predict(X_val_fold_scaled)
            y_val_prob = svm_model.predict_proba(X_val_fold_scaled)[:, 1]  # Probabilities for positive class

            # Record validation metrics
            accuracy_scores.append(accuracy_score(y_val_fold, y_val_pred))
            precision_scores.append(precision_score(y_val_fold, y_val_pred))
            recall_scores.append(recall_score(y_val_fold, y_val_pred))
            f1_scores.append(f1_score(y_val_fold, y_val_pred))
            auroc_scores.append(roc_auc_score(y_val_fold, y_val_prob))
            auprc_scores.append(average_precision_score(y_val_fold, y_val_prob))

        # Calculate mean and standard deviation of AUROC
        auroc_mean = np.mean(auroc_scores)
        auroc_std = np.std(auroc_scores)

        # Check if this is the best AUROC so far
        if auroc_mean > best_auroc_mean:
            best_auroc_mean = auroc_mean
            best_auroc_std = auroc_std
            best_metrics = {
                'accuracy': (np.mean(accuracy_scores), np.std(accuracy_scores)),
                'precision': (np.mean(precision_scores), np.std(precision_scores)),
                'recall': (np.mean(recall_scores), np.std(recall_scores)),
                'f1_score': (np.mean(f1_scores), np.std(f1_scores)),
                'auroc': (auroc_mean, auroc_std),
                'auprc': (np.mean(auprc_scores), np.std(auprc_scores))
            }
            best_hyperparams = {'C': C, 'gamma': gamma}

        # Report mean and standard deviation of cross-validation metrics
        # print(f"\tAccuracy: Mean = {np.mean(accuracy_scores):.4f}, Std = {np.std(accuracy_scores):.4f}")
        # print(f"\tPrecision: Mean = {np.mean(precision_scores):.4f}, Std = {np.std(precision_scores):.4f}")
        # print(f"\tRecall: Mean = {np.mean(recall_scores):.4f}, Std = {np.std(recall_scores):.4f}")
        # print(f"\tF1-Score: Mean = {np.mean(f1_scores):.4f}, Std = {np.std(f1_scores):.4f}")
        # print(f"\tAUROC: Mean = {auroc_mean:.4f}, Std = {auroc_std:.4f}")
        # print(f"\tAUPRC: Mean = {np.mean(auprc_scores):.4f}, Std = {np.std(auprc_scores):.4f}")

# Print the best results
print(f"Best AUROC: {best_metrics['auroc'][0]:.4f} ({best_metrics['auroc'][1]:.4f})")
print(f"Best hyperparameters: C = {best_hyperparams['C']}, Gamma = {best_hyperparams['gamma']}")
print(f"Accuracy: Mean = {best_metrics['accuracy'][0]:.4f} ({best_metrics['accuracy'][1]:.4f})")
print(f"Precision: Mean = {best_metrics['precision'][0]:.4f} ({best_metrics['precision'][1]:.4f})")
print(f"Recall: Mean = {best_metrics['recall'][0]:.4f} ({best_metrics['recall'][1]:.4f})")
print(f"F1-Score: Mean = {best_metrics['f1_score'][0]:.4f} ({best_metrics['f1_score'][1]:.4f})")
print(f"AUROC: Mean = {best_metrics['auroc'][0]:.4f} ({best_metrics['auroc'][1]:.4f})")
print(f"AUPRC: Mean = {best_metrics['auprc'][0]:.4f} ({best_metrics['auprc'][1]:.4f})")


Regularization strength (C): 0.01, Gamma: scale
Regularization strength (C): 0.01, Gamma: auto
Regularization strength (C): 0.1, Gamma: scale
Regularization strength (C): 0.1, Gamma: auto
Regularization strength (C): 1, Gamma: scale
Regularization strength (C): 1, Gamma: auto
Regularization strength (C): 10, Gamma: scale
Regularization strength (C): 10, Gamma: auto
Regularization strength (C): 100, Gamma: scale
Regularization strength (C): 100, Gamma: auto
Best AUROC: 0.7145 (0.0401)
Best hyperparameters: C = 1, Gamma = scale
Accuracy: Mean = 0.6458 (0.0520)
Precision: Mean = 0.6533 (0.0490)
Recall: Mean = 0.6308 (0.0636)
F1-Score: Mean = 0.6417 (0.0558)
AUROC: Mean = 0.7145 (0.0401)
AUPRC: Mean = 0.7098 (0.0263)


In [32]:
# After determining the best hyperparameters 'C' and 'gamma' from cross-validation:
best_C = best_hyperparams['C']
best_gamma = best_hyperparams['gamma']

# Normalize the entire development set and the test set
scaler = StandardScaler()
X_dev_scaled = scaler.fit_transform(X_dev)
X_test_scaled = scaler.transform(X_test)

# Fit the SVM model on the entire development set using the best hyperparameters
svm_model = SVC(C=best_C, kernel='rbf', gamma=best_gamma, probability=True, random_state=42)
svm_model.fit(X_dev_scaled, y_dev)

# Predict on the test set
y_test_pred = svm_model.predict(X_test_scaled)
y_test_prob = svm_model.predict_proba(X_test_scaled)[:, 1]  # Probabilities for the positive class

# Evaluate the test set metrics
accuracy_test = accuracy_score(y_test, y_test_pred)
precision_test = precision_score(y_test, y_test_pred)
recall_test = recall_score(y_test, y_test_pred)
f1_test = f1_score(y_test, y_test_pred)
auroc_test = roc_auc_score(y_test, y_test_prob)
auprc_test = average_precision_score(y_test, y_test_prob)

# Print the test set performance
print(f"Accuracy on test set: {accuracy_test:.4f}")
print(f"Precision on test set: {precision_test:.4f}")
print(f"Recall on test set: {recall_test:.4f}")
print(f"F1-Score on test set: {f1_test:.4f}")
print(f"AUROC on test set: {auroc_test:.4f}")
print(f"AUPRC on test set: {auprc_test:.4f}")


Accuracy on test set: 0.7222
Precision on test set: 0.7412
Recall on test set: 0.6923
F1-Score on test set: 0.7159
AUROC on test set: 0.7329
AUPRC on test set: 0.7232


# Poly SVM

In [35]:
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, average_precision_score, roc_curve
from sklearn.preprocessing import StandardScaler
import numpy as np

In [36]:
# Initialize StratifiedKFold for cross-validation
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Define hyperparameters to tune for the Polynomial kernel
C_values = [0.01, 0.1, 1, 10, 100]
gammas = ['scale', 'auto']  # Gamma options for the polynomial kernel
degrees = [2, 3, 4]  # Polynomial degrees to explore

# Track the best results
best_auroc_mean = 0
best_auroc_std = 0
best_metrics = None
best_hyperparams = None

# Perform grid search over C, gamma, and degree
for C in C_values:
    for gamma in gammas:
        for degree in degrees:
            print(f"Regularization strength (C): {C}, Gamma: {gamma}, Degree: {degree}")
            
            # Initialize lists to store metrics for each fold
            accuracy_scores = []
            precision_scores = []
            recall_scores = []
            f1_scores = []
            auroc_scores = []
            auprc_scores = []

            # Perform 5-fold cross-validation
            for train_index, val_index in kf.split(X_dev, y_dev):
                # Get train and validation data
                X_train_fold, X_val_fold = X_dev.iloc[train_index], X_dev.iloc[val_index]
                y_train_fold, y_val_fold = y_dev.iloc[train_index], y_dev.iloc[val_index]

                # Normalize the training set and apply the same parameters to validation
                scaler = StandardScaler()
                X_train_fold_scaled = scaler.fit_transform(X_train_fold)
                X_val_fold_scaled = scaler.transform(X_val_fold)

                # Fit SVM model with Polynomial kernel
                svm_model = SVC(C=C, kernel='poly', gamma=gamma, degree=degree, coef0=1, probability=True, random_state=42)
                svm_model.fit(X_train_fold_scaled, y_train_fold)

                # Predict on validation set
                y_val_pred = svm_model.predict(X_val_fold_scaled)
                y_val_prob = svm_model.predict_proba(X_val_fold_scaled)[:, 1]  # Probabilities for positive class

                # Record validation metrics
                accuracy_scores.append(accuracy_score(y_val_fold, y_val_pred))
                precision_scores.append(precision_score(y_val_fold, y_val_pred))
                recall_scores.append(recall_score(y_val_fold, y_val_pred))
                f1_scores.append(f1_score(y_val_fold, y_val_pred))
                auroc_scores.append(roc_auc_score(y_val_fold, y_val_prob))
                auprc_scores.append(average_precision_score(y_val_fold, y_val_prob))

            # Calculate mean and standard deviation of AUROC
            auroc_mean = np.mean(auroc_scores)
            auroc_std = np.std(auroc_scores)

            # Check if this is the best AUROC so far
            if auroc_mean > best_auroc_mean:
                best_auroc_mean = auroc_mean
                best_auroc_std = auroc_std
                best_metrics = {
                    'accuracy': (np.mean(accuracy_scores), np.std(accuracy_scores)),
                    'precision': (np.mean(precision_scores), np.std(precision_scores)),
                    'recall': (np.mean(recall_scores), np.std(recall_scores)),
                    'f1_score': (np.mean(f1_scores), np.std(f1_scores)),
                    'auroc': (auroc_mean, auroc_std),
                    'auprc': (np.mean(auprc_scores), np.std(auprc_scores))
                }
                best_hyperparams = {'C': C, 'gamma': gamma, 'degree': degree}

            # Report mean and standard deviation of cross-validation metrics
            # print(f"\tAccuracy:  {np.mean(accuracy_scores):.4f} ({np.std(accuracy_scores):.4f})")
            # print(f"\tPrecision: {np.mean(precision_scores):.4f} ({np.std(precision_scores):.4f})")
            # print(f"\tRecall:    {np.mean(recall_scores):.4f} ({np.std(recall_scores):.4f})")
            # print(f"\tF1-Score:  {np.mean(f1_scores):.4f} ({np.std(f1_scores):.4f})")
            # print(f"\tAUROC:     {auroc_mean:.4f} ({auroc_std:.4f})")
            # print(f"\tAUPRC:     {np.mean(auprc_scores):.4f} ({np.std(auprc_scores):.4f})")

# Print the best results
print(f"Best AUROC: {best_metrics['auroc'][0]:.4f} ({best_metrics['auroc'][1]:.4f})")
print(f"Best hyperparameters: C = {best_hyperparams['C']}, Gamma = {best_hyperparams['gamma']}, Degree = {best_hyperparams['degree']}")
print(f"Accuracy: Mean = {best_metrics['accuracy'][0]:.4f} ({best_metrics['accuracy'][1]:.4f})")
print(f"Precision: Mean = {best_metrics['precision'][0]:.4f} ({best_metrics['precision'][1]:.4f})")
print(f"Recall: Mean = {best_metrics['recall'][0]:.4f} ({best_metrics['recall'][1]:.4f})")
print(f"F1-Score: Mean = {best_metrics['f1_score'][0]:.4f} ({best_metrics['f1_score'][1]:.4f})")
print(f"AUROC: Mean = {best_metrics['auroc'][0]:.4f} ({best_metrics['auroc'][1]:.4f})")
print(f"AUPRC: Mean = {best_metrics['auprc'][0]:.4f} ({best_metrics['auprc'][1]:.4f})")


Regularization strength (C): 0.01, Gamma: scale, Degree: 2
Regularization strength (C): 0.01, Gamma: scale, Degree: 3
Regularization strength (C): 0.01, Gamma: scale, Degree: 4
Regularization strength (C): 0.01, Gamma: auto, Degree: 2
Regularization strength (C): 0.01, Gamma: auto, Degree: 3
Regularization strength (C): 0.01, Gamma: auto, Degree: 4
Regularization strength (C): 0.1, Gamma: scale, Degree: 2
Regularization strength (C): 0.1, Gamma: scale, Degree: 3
Regularization strength (C): 0.1, Gamma: scale, Degree: 4
Regularization strength (C): 0.1, Gamma: auto, Degree: 2
Regularization strength (C): 0.1, Gamma: auto, Degree: 3
Regularization strength (C): 0.1, Gamma: auto, Degree: 4
Regularization strength (C): 1, Gamma: scale, Degree: 2
Regularization strength (C): 1, Gamma: scale, Degree: 3
Regularization strength (C): 1, Gamma: scale, Degree: 4
Regularization strength (C): 1, Gamma: auto, Degree: 2
Regularization strength (C): 1, Gamma: auto, Degree: 3
Regularization strength (C

In [37]:
# After determining the best hyperparameters 'C', 'gamma', and 'degree' from cross-validation:
best_C = best_hyperparams['C']
best_gamma = best_hyperparams['gamma']
best_degree = best_hyperparams['degree']

# Normalize the entire development set and the test set
scaler = StandardScaler()
X_dev_scaled = scaler.fit_transform(X_dev)
X_test_scaled = scaler.transform(X_test)

# Fit the SVM model on the entire development set using the best hyperparameters
svm_model = SVC(C=best_C, kernel='poly', gamma=best_gamma, degree=best_degree, coef0=1, probability=True, random_state=42)
svm_model.fit(X_dev_scaled, y_dev)

# Predict on the test set
y_test_pred = svm_model.predict(X_test_scaled)
y_test_prob = svm_model.predict_proba(X_test_scaled)[:, 1]  # Probabilities for the positive class

# Evaluate the test set metrics
accuracy_test = accuracy_score(y_test, y_test_pred)
precision_test = precision_score(y_test, y_test_pred)
recall_test = recall_score(y_test, y_test_pred)
f1_test = f1_score(y_test, y_test_pred)
auroc_test = roc_auc_score(y_test, y_test_prob)
auprc_test = average_precision_score(y_test, y_test_prob)

# Print the test set performance
print(f"Accuracy on test set: {accuracy_test:.4f}")
print(f"Precision on test set: {precision_test:.4f}")
print(f"Recall on test set: {recall_test:.4f}")
print(f"F1-Score on test set: {f1_test:.4f}")
print(f"AUROC on test set: {auroc_test:.4f}")
print(f"AUPRC on test set: {auprc_test:.4f}")

Accuracy on test set: 0.6944
Precision on test set: 0.7143
Recall on test set: 0.6593
F1-Score on test set: 0.6857
AUROC on test set: 0.7602
AUPRC on test set: 0.7481


# MLP

In [42]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, average_precision_score
from sklearn.preprocessing import StandardScaler
import numpy as np

# Initialize StratifiedKFold for cross-validation
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Define hyperparameters to tune for MLP
hidden_layer_sizes = [(6,)]  # Hidden layer size fixed at 6
alphas = [0.0001, 0.001, 0.01, 0.1]  # Regularization parameter (alpha)
learning_rates = ['constant', 'adaptive']  # Learning rate options
max_iters = [50, 100, 200, 300]  # Max iterations for convergence

# Track the best results
best_auroc_mean = 0
best_auroc_std = 0
best_metrics = None
best_hyperparams = None

# Perform grid search over alphas, learning_rates, and max_iters
for alpha in alphas:
    for learning_rate in learning_rates:
        for max_iter in max_iters:
            print(f"Alpha: {alpha}, Learning Rate: {learning_rate}, Max Iterations: {max_iter}")
            
            # Initialize lists to store metrics for each fold
            accuracy_scores = []
            precision_scores = []
            recall_scores = []
            f1_scores = []
            auroc_scores = []
            auprc_scores = []

            # Perform 5-fold cross-validation
            for train_index, val_index in kf.split(X_dev, y_dev):
                # Get train and validation data
                X_train_fold, X_val_fold = X_dev.iloc[train_index], X_dev.iloc[val_index]
                y_train_fold, y_val_fold = y_dev.iloc[train_index], y_dev.iloc[val_index]

                # Normalize the training set and apply the same params to validation
                scaler = StandardScaler()
                X_train_fold_scaled = scaler.fit_transform(X_train_fold)
                X_val_fold_scaled = scaler.transform(X_val_fold)

                # Fit MLP model
                mlp_model = MLPClassifier(hidden_layer_sizes=(6,), alpha=alpha, learning_rate=learning_rate, 
                                          max_iter=max_iter, random_state=42)
                mlp_model.fit(X_train_fold_scaled, y_train_fold)

                # Predict on validation set
                y_val_pred = mlp_model.predict(X_val_fold_scaled)
                y_val_prob = mlp_model.predict_proba(X_val_fold_scaled)[:, 1]  # Probabilities for positive class

                # Record validation metrics
                accuracy_scores.append(accuracy_score(y_val_fold, y_val_pred))
                precision_scores.append(precision_score(y_val_fold, y_val_pred))
                recall_scores.append(recall_score(y_val_fold, y_val_pred))
                f1_scores.append(f1_score(y_val_fold, y_val_pred))
                auroc_scores.append(roc_auc_score(y_val_fold, y_val_prob))
                auprc_scores.append(average_precision_score(y_val_fold, y_val_prob))

            # Calculate mean and standard deviation of AUROC
            auroc_mean = np.mean(auroc_scores)
            auroc_std = np.std(auroc_scores)

            # Check if this is the best AUROC so far
            if auroc_mean > best_auroc_mean:
                best_auroc_mean = auroc_mean
                best_auroc_std = auroc_std
                best_metrics = {
                    'accuracy': (np.mean(accuracy_scores), np.std(accuracy_scores)),
                    'precision': (np.mean(precision_scores), np.std(precision_scores)),
                    'recall': (np.mean(recall_scores), np.std(recall_scores)),
                    'f1_score': (np.mean(f1_scores), np.std(f1_scores)),
                    'auroc': (auroc_mean, auroc_std),
                    'auprc': (np.mean(auprc_scores), np.std(auprc_scores))
                }
                best_hyperparams = {'alpha': alpha, 'learning_rate': learning_rate, 'max_iter': max_iter}

            # Report mean and standard deviation of cross-validation metrics
#             print(f"\tAccuracy:  {np.mean(accuracy_scores):.4f} ({np.std(accuracy_scores):.4f})")
#             print(f"\tPrecision: {np.mean(precision_scores):.4f} ({np.std(precision_scores):.4f})")
#             print(f"\tRecall:    {np.mean(recall_scores):.4f} ({np.std(recall_scores):.4f})")
#             print(f"\tF1-Score:  {np.mean(f1_scores):.4f} ({np.std(f1_scores):.4f})")
#             print(f"\tAUROC:     {auroc_mean:.4f} ({auroc_std:.4f})")
#             print(f"\tAUPRC:     {np.mean(auprc_scores):.4f} ({np.std(auprc_scores):.4f})")

# Print the best results
print(f"Best AUROC: {best_metrics['auroc'][0]:.4f} ({best_metrics['auroc'][1]:.4f})")
print(f"Best hyperparameters: Alpha = {best_hyperparams['alpha']}, Learning Rate = {best_hyperparams['learning_rate']}, Max Iterations = {best_hyperparams['max_iter']}")
print(f"Accuracy: Mean = {best_metrics['accuracy'][0]:.4f} ({best_metrics['accuracy'][1]:.4f})")
print(f"Precision: Mean = {best_metrics['precision'][0]:.4f} ({best_metrics['precision'][1]:.4f})")
print(f"Recall: Mean = {best_metrics['recall'][0]:.4f} ({best_metrics['recall'][1]:.4f})")
print(f"F1-Score: Mean = {best_metrics['f1_score'][0]:.4f} ({best_metrics['f1_score'][1]:.4f})")
print(f"AUROC: Mean = {best_metrics['auroc'][0]:.4f} ({best_metrics['auroc'][1]:.4f})")
print(f"AUPRC: Mean = {best_metrics['auprc'][0]:.4f} ({best_metrics['auprc'][1]:.4f})")


Alpha: 0.0001, Learning Rate: constant, Max Iterations: 50
	Accuracy:  0.5917 (0.0296)
	Precision: 0.5939 (0.0257)
	Recall:    0.5978 (0.0520)
	F1-Score:  0.5953 (0.0369)
	AUROC:     0.6163 (0.0314)
	AUPRC:     0.6263 (0.0260)
Alpha: 0.0001, Learning Rate: constant, Max Iterations: 100


/Users/oliviazhang/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/oliviazhang/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/oliviazhang/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/oliviazhang/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(
/Use

	Accuracy:  0.6222 (0.0336)
	Precision: 0.6352 (0.0381)
	Recall:    0.5923 (0.0242)
	F1-Score:  0.6128 (0.0299)
	AUROC:     0.6550 (0.0221)
	AUPRC:     0.6548 (0.0221)
Alpha: 0.0001, Learning Rate: constant, Max Iterations: 200


/Users/oliviazhang/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/oliviazhang/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/oliviazhang/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/oliviazhang/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


	Accuracy:  0.6431 (0.0293)
	Precision: 0.6504 (0.0298)
	Recall:    0.6337 (0.0561)
	F1-Score:  0.6407 (0.0352)
	AUROC:     0.6741 (0.0145)
	AUPRC:     0.6693 (0.0206)
Alpha: 0.0001, Learning Rate: constant, Max Iterations: 300


...

/Users/oliviazhang/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/oliviazhang/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/oliviazhang/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/oliviazhang/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


	Accuracy:  0.6458 (0.0304)
	Precision: 0.6541 (0.0312)
	Recall:    0.6337 (0.0561)
	F1-Score:  0.6425 (0.0359)
	AUROC:     0.6737 (0.0146)
	AUPRC:     0.6674 (0.0229)
Alpha: 0.1, Learning Rate: adaptive, Max Iterations: 300


/Users/oliviazhang/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/oliviazhang/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


	Accuracy:  0.6444 (0.0391)
	Precision: 0.6496 (0.0415)
	Recall:    0.6447 (0.0680)
	F1-Score:  0.6453 (0.0439)
	AUROC:     0.6825 (0.0237)
	AUPRC:     0.6730 (0.0239)
Best AUROC: 0.6833 (0.0255)
Best hyperparameters: Alpha = 0.0001, Learning Rate = constant, Max Iterations = 300
Accuracy: Mean = 0.6458 (0.0407)
Precision: Mean = 0.6504 (0.0419)
Recall: Mean = 0.6475 (0.0725)
F1-Score: Mean = 0.6470 (0.0463)
AUROC: Mean = 0.6833 (0.0255)
AUPRC: Mean = 0.6735 (0.0235)


/Users/oliviazhang/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/oliviazhang/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


In [43]:
# After determining the best hyperparameters from cross-validation:
best_alpha = best_hyperparams['alpha']
best_learning_rate = best_hyperparams['learning_rate']
best_max_iter = best_hyperparams['max_iter']

# Normalize the entire development set and the test set
scaler = StandardScaler()
X_dev_scaled = scaler.fit_transform(X_dev)
X_test_scaled = scaler.transform(X_test)

# Fit the MLP model on the entire development set using the best hyperparameters
mlp_model = MLPClassifier(hidden_layer_sizes=(6,), alpha=best_alpha, learning_rate=best_learning_rate, 
                          max_iter=best_max_iter, random_state=42)
mlp_model.fit(X_dev_scaled, y_dev)

# Predict on the test set
y_test_pred = mlp_model.predict(X_test_scaled)
y_test_prob = mlp_model.predict_proba(X_test_scaled)[:, 1]  # Probabilities for the positive class

# Evaluate the test set metrics
accuracy_test = accuracy_score(y_test, y_test_pred)
precision_test = precision_score(y_test, y_test_pred)
recall_test = recall_score(y_test, y_test_pred)
f1_test = f1_score(y_test, y_test_pred)
auroc_test = roc_auc_score(y_test, y_test_prob)
auprc_test = average_precision_score(y_test, y_test_prob)

# Print the test set performance
print(f"Accuracy on test set: {accuracy_test:.4f}")
print(f"Precision on test set: {precision_test:.4f}")
print(f"Recall on test set: {recall_test:.4f}")
print(f"F1-Score on test set: {f1_test:.4f}")
print(f"AUROC on test set: {auroc_test:.4f}")
print(f"AUPRC on test set: {auprc_test:.4f}")


Accuracy on test set: 0.7167
Precision on test set: 0.7222
Recall on test set: 0.7143
F1-Score on test set: 0.7182
AUROC on test set: 0.7527
AUPRC on test set: 0.7667


/Users/oliviazhang/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
